### New install

In [1]:
!pip install --upgrade jupyterlab
!pip install --upgrade jupyter
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


## Imports

In [2]:
import logging
import sys
sys.path.append('../../src')
import timm
import torch
import torch.nn as nn
import torchaudio.transforms as AT
import torchvision.transforms as VT
from nnAudio import features

import config

Created directory: /dli/data/plots
Created directory: /dli/data/features/MED
Created directory: /dli/models


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [3]:


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [4]:
def load_model(model_file,model = timm.create_model('convnext_small',pretrained=True, num_classes=2, in_chans=1, drop_path_rate=0.05, global_pool='max', drop_rate=0.05)):
    model = model
    state_dict = torch.load(model_file)
    model.load_state_dict(state_dict)
    model.train()
    return model

In [5]:

model = timm.create_model('convnext_small',pretrained=True, num_classes=2, in_chans=1, drop_path_rate=0.05, global_pool='max', drop_rate=0.05)


## Load the base fp-32 model

In [10]:
!pwd

/dli/task


In [18]:
import os
filename = os.path.join("models","model_med14_2023_11_11_03_44_30.pth")

#state_dict = torch.load(filename)
model_fp32  = load_model(filename)





In [11]:
#let's try to save the float model


In [19]:
def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

## Quantization starts here

In [21]:
model_fp32.eval()
model_fp32.qconfig = torch.ao.quantization.get_default_qconfig('x86')
from torch.quantization import convert


In [23]:
model_fp32_prepared = torch.ao.quantization.prepare(model_fp32)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


In [15]:
model_ft_tuned_new.cpu()

ConvNeXt(
  (stem): Sequential(
    (0): Conv2d(1, 96, kernel_size=(4, 4), stride=(4, 4))
    (1): LayerNorm2d((96,), eps=1e-06, elementwise_affine=True)
  )
  (stages): Sequential(
    (0): ConvNeXtStage(
      (downsample): Identity()
      (blocks): Sequential(
        (0): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
          (norm): LayerNorm((96,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=96, out_features=384, bias=True)
            (act): GELU()
            (drop1): Dropout(p=0.0, inplace=False)
            (norm): Identity()
            (fc2): Linear(in_features=384, out_features=96, bias=True)
            (drop2): Dropout(p=0.0, inplace=False)
          )
          (shortcut): Identity()
          (drop_path): Identity()
        )
        (1): ConvNeXtBlock(
          (conv_dw): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)


In [24]:
model_int8 = torch.ao.quantization.convert(model_fp32_prepared)

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:1207: UserWarning: must run observer before calling calculate_qparams.                                    Returning default scale and zero point 
  warnings.warn(


In [10]:
#torch.jit.script(model_int8 ,"convnext_ft_quant_int8.pt")

/opt/conda/lib/python3.8/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


RecursiveScriptModule(
  original_name=ConvNeXt
  (stem): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Conv2d)
    (1): RecursiveScriptModule(original_name=LayerNorm2d)
  )
  (stages): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(
      original_name=ConvNeXtStage
      (downsample): RecursiveScriptModule(original_name=Identity)
      (blocks): RecursiveScriptModule(
        original_name=Sequential
        (0): RecursiveScriptModule(
          original_name=ConvNeXtBlock
          (conv_dw): RecursiveScriptModule(original_name=Conv2d)
          (norm): RecursiveScriptModule(original_name=LayerNorm)
          (mlp): RecursiveScriptModule(
            original_name=Mlp
            (fc1): RecursiveScriptModule(original_name=Linear)
            (act): RecursiveScriptModule(original_name=GELU)
            (drop1): RecursiveScriptModule(original_name=Dropout)
            (norm): RecursiveScriptModule(o

In [9]:
#model_int8_new_t_no_qnt_layer = torch.quantization.convert(model_fp32_prepared.to('cpu').eval())

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/utils.py:317: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(


In [25]:
torch.jit.save(torch.jit.script(model_int8), "convnext_ft_quant_int8.pt")